In [ ]:
def single_baseline(elecSeg, img_slice, baseStruct, root, analysedPath, labelsdf):
#     struct="crtx"
    print("Baseline structure: "+baseStruct)
    base_seg=labelsdf["Labels"][labelsdf["Anatomical Regions"]==baseStruct].values[0]
    baseline=get_baseline_vals(img_slice, base_seg, coronal1seg, coronal1data)        
    segmentation_mask=coronal1seg[:,:,img_slice]

#     segname="mono"
    elec_segs=labelsdf["Labels"][labelsdf["Anatomical Regions"]==elecSeg].values
    print("Segment of interest: "+elecSeg)
    save_path=root+analysedPath+elecSeg+"-"+baseStruct+"_base/"
    dpi=1000
    savefigs=True
    zthr=-1.95

    if not os.path.exists(save_path):
        os.mkdir(save_path)

    num_echos=coronal1data.shape[-1]
    mean_intensities=np.zeros([len(elec_segs),num_echos])

    px_count=np.zeros((len(elec_segs),num_echos))
    px_intensities=[]

    for j,seg in enumerate(elec_segs):
        tmpvals=[]
        for echo in range(num_echos):
            img=coronal1data[:,:,img_slice,echo]
            imgz=img_zscore(img,baseline,echo)
    #         imgz=(img-np.mean(baseline[:,echo]))/np.std(baseline[:,echo])

            imgz=imgz*(segmentation_mask==seg)
            vals_z=imgz[segmentation_mask==seg]
            ind=np.where(imgz<zthr)
            px_count[j, echo]=np.sum(vals_z<zthr)
            vals=img[ind]
            tmpvals.append(vals)
            mean_intensities[j,echo]=np.mean(vals)

            plt.imshow(img, cmap='gray')
            plt.scatter(ind[1],ind[0], c='r', s=0.01)
            if savefigs:
                filename=elecSeg+str(seg)+"echo"+str(echo)+".png"
                plt.savefig(save_path+filename, dpi=dpi)
            plt.show()

        px_intensities.append(tmpvals)

In [ ]:
def weighted_pixel_index(img, index, xind_w, xind_coef, yind_w, yind_coef):
    """
    Weigh the pixel indices with pixel intensities
    
    """
    xind_w.append(index[1]*((255-img[index[0],index[1]])/255))
    xind_coef.append((255-img[index[0],index[1]])/255)
                
    yind_w.append(index[0]*((255-img[index[0],index[1]])/255))
    yind_coef.append((255-img[index[0],index[1]])/255)
    
    return xind_w, xind_coef, yind_w, yind_coef

In [ ]:
def get_length_legacy(ind):
    indx, indy = ind
    allIdx=np.concatenate((np.reshape(indx,(-1,1)),np.reshape(indy,(-1,1))),axis=1)
    candidates = allIdx[spatial.ConvexHull(allIdx).vertices]
    # get distances between each pair of candidate points
    dist_mat = spatial.distance_matrix(candidates, candidates)

    # get indices of candidates that are furthest apart
    i, j = np.unravel_index(dist_mat.argmax(), dist_mat.shape)

#     plt.scatter(allIdx[:,1],allIdx[:,0])
#     plt.scatter(candidates[:,1],candidates[:,0], linewidths=1)
#     plt.scatter(candidates[i][1],candidates[i][0])
#     plt.scatter(candidates[j][1],candidates[j][0])

    p1=candidates[i][:]
    p2=candidates[j][:]
    length=np.linalg.norm((p1-p2))
    return p1, p2, length    